In [1]:
##
import os
import sys
import warnings
import enum
from enum import Enum
import matplotlib
from pathlib import Path
from typing import Callable

##
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

##
from finrl import config as finrl_config
from finrl.main import check_and_make_directories
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.config import (
    TEST_END_DATE,
    TRAINED_MODEL_DIR,
)
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.plot import backtest_stats, convert_daily_return_to_pyfolio_ts, get_baseline

##
import yfinance as yf

##
from meta.data_processors.yahoofinance import Yahoofinance

/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/tensorboard/compat/proto/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/tensorboard/compat/proto/tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors 

In [2]:
def config():
    # #
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    # os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # INFO, WARN are ignored and only ERROR messages will be printed

    # #
    matplotlib.use("Agg")

    # #
    check_and_make_directories(
        [
            finrl_config.DATA_SAVE_DIR,
            finrl_config.TRAINED_MODEL_DIR,
            finrl_config.TENSORBOARD_LOG_DIR,
            finrl_config.RESULTS_DIR,
        ]
    )


config()

In [3]:
##
import numpy as np
import pandas as pd
from pyfolio import timeseries
import tqdm

##
from rl.data import DataTechnicalAnalysis, DataFundamentalAnalysis, DataBase
from common.utils import now_time
from configuration.settings import ProjectDir

In [4]:
def print_info(df: pd.DataFrame):
    ##
    print(f"{df.shape=}")
    print(f"Start date: {df['date'].iloc[0]}")
    print(f"End date: {df['date'].iloc[-1]}")
    print(f"Stocks: {df['tic'].unique().size}")

    ##
    data_grouped_by_date = df.groupby(by=["date"])
    grouped_data_size = data_grouped_by_date.size()
    if grouped_data_size.nunique() == 1:
        print(f"OK: All dates have same row count: {grouped_data_size.unique()}")
    else:
        raise ValueError(f"Different size for each date: {grouped_data_size.unique()}")


def check_dir(_filename: Path):
    dir = _filename.parent
    if os.path.isdir(dir):
        print(f"OK: Dir exist for: {_filename}")
    else:
        raise NotADirectoryError(f"ERROR: Dir does not exist {dir}")


class FileType(Enum):
    JSON = enum.auto
    CSV = enum.auto


def save_dataset(dataframe: pd.DataFrame, type: FileType, _filename: Path):
    print(f"Saving data into: {_filename}")
    if type == FileType.JSON:
        dataframe.to_json(_filename.as_posix())
    elif type == FileType.CSV:
        dataframe.to_csv(_filename)
    else:
        raise ValueError(f"type: FileType must be CSV or JSON.")
    print(f"Data saved to json: {_filename}")

In [5]:
prj_dir = ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing"))
_TRAIN_DATA_START = "2010-01-01"
_TRAIN_DATA_END = "2021-12-31"
_TEST_DATA_START = "2021-01-01"
_TEST_DATA_END = "2021-12-31"
tickers = DOW_30_TICKER

In [6]:
print(tickers.__len__())
print(tickers)
data_processor = DataBase(
    data_source="yahoofinance", start_date=_TRAIN_DATA_START, end_date=_TEST_DATA_END, time_interval="1d"
)

30
['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [7]:
data_processor.download_data(ticker_list=tickers)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [8]:
print(data_processor.dataframe.shape)
data_processor.dataframe.head()
# print_info(data_processor.dataframe)

(88283, 9)


,date,open,high,low,close,adjusted_close,volume,tic,day
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.515213,493729600,AAPL,0
1,2010-01-04,56.630001,57.869999,56.560001,57.720001,43.267181,5277400,AMGN,0
2,2010-01-04,40.810001,41.099998,40.389999,40.919998,33.792698,6894300,AXP,0
3,2010-01-04,55.720001,56.389999,54.799999,56.180000,43.777550,6186700,BA,0
4,2010-01-04,57.650002,59.189999,57.509998,58.549999,41.353191,7325600,CAT,0


In [9]:
def get_missing_counts(df: pd.DataFrame):
    grouped_by_df = df.groupby(by=["date"])
    grouped_by_size = grouped_by_df.size()
    counts = pd.Series([])
    for i in grouped_by_size.unique():
        occurences = (grouped_by_size == i).sum()
        counts = counts.append(pd.Series(occurences))
        print(f"{i} : {occurences}")


get_missing_counts(data_processor.dataframe)

29 : 2317
30 : 703


In [10]:
def get_missing_tic(df: pd.DataFrame, date: str) -> list:
    df_date = df[df["date"] == date]
    missing_tics: set = set(tickers).difference(set(df_date["tic"].values))
    missing_tics: list = list(missing_tics)
    return [missing_tics]


def get_previous_value(df: pd.DataFrame, date: str, missing_tic: str):
    df_before_date_binary = df["date"] < date
    df_before_date = df[df_before_date_binary]
    previous_value_of_missing_tic = df_before_date["tic"] == missing_tic[0]
    df_missing_tic_previous_value = df_before_date[previous_value_of_missing_tic]
    if df_missing_tic_previous_value.empty:
        return None
    else:
        return df_missing_tic_previous_value.iloc[-1]


def get_following_value(df: pd.DataFrame, date: str, missing_tic: str):
    df_before_date_binary = df["date"] > date
    df_before_date = df[df_before_date_binary]
    previous_value_of_missing_tic_binary = df_before_date["tic"] == missing_tic[0]
    df_missing_tic_previous_value = df_before_date[previous_value_of_missing_tic_binary]
    if df_missing_tic_previous_value.empty:
        return None
    else:
        return df_missing_tic_previous_value.iloc[0]


def fill_missed_tic_gaps(df: pd.DataFrame) -> pd.DataFrame:
    grouped_by_df = data_processor.dataframe.groupby(by=["date"])
    grouped_by_size = grouped_by_df.size()
    max_size = grouped_by_size.max()
    # print(grouped_by_size.unique())

    for date in tqdm.tqdm(
        grouped_by_size[grouped_by_size < max_size].index
    ):  # for each date where some tics are missed
        ##
        tics = get_missing_tic(df, date)  # TODO: Improve performance 1/3 time is here

        ##
        for tic in tics:
            ##
            filled_value = get_previous_value(df, date, missing_tic=tic)  # TODO: Improve performance 1/3 time is here
            filled_value = filled_value if filled_value is not None else get_following_value(df, date, missing_tic=tic)

            ##
            if filled_value is None:
                raise ValueError("None value of filling value")
            else:
                filled_value["date"] = date  # update date we want to fill the gap
                df = df.append(filled_value)  # TODO: Improve performance 1/3 time is here

    return df


def get_difference_in_timeframe(df: pd.DataFrame):
    grouped_by_df = df.groupby(by=["date"])
    grouped_by_size = grouped_by_df.size()
    return grouped_by_size


# % load_ext line_profiler

# % lprun -f fill_missed_tic_gaps df_filled_gaps = fill_missed_tic_gaps(data_processor.dataframe)

df_filled_gaps = fill_missed_tic_gaps(data_processor.dataframe)
# print(get_difference_in_timeframe(df_filled_gaps).unique())

100%|██████████| 2317/2317 [00:52<00:00, 44.31it/s]


In [11]:
filename = data_processor.get_filename(prj_dir=prj_dir, name="dji30_ta")
check_dir(filename)

# %load_ext line_profiler
# %lprun -f save_dataset save_dataset()

OK: Dir exist for: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-15T02-08-46.json


In [12]:
df_filled_gaps.shape

(90600, 9)

In [13]:
fe = FeatureEngineer(use_technical_indicator=True, use_turbulence=False, user_defined_feature=False)
data_features: pd.DataFrame = fe.preprocess_data(df_filled_gaps.copy())

Successfully added technical indicators


In [16]:
data_features = data_features.sort_values(["date", "tic"]).reset_index(drop=True)

In [17]:
data_features.shape

(90600, 17)

In [18]:
save_dataset(dataframe=data_features, type=FileType.JSON, _filename=filename)

Saving data into: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-15T02-08-46.json
Data saved to json: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-15T02-08-46.json


In [52]:
def add_covariance_as_nested_states(df: pd.DataFrame) -> pd.DataFrame:
    ##
    df = df.sort_values(["date", "tic"], ignore_index=True)
    df.index = df.date.factorize()[0]

    cov_list = []
    return_list = []

    # look back is one year
    lookback = 252
    for i in tqdm.tqdm(range(lookback, len(df.index.unique()))):
        data_lookback = df.loc[i - lookback : i, :]
        price_lookback = data_lookback.pivot_table(index="date", columns="tic", values="close")
        return_lookback = price_lookback.pct_change().dropna()
        covs = return_lookback.cov().values

        #
        return_list.append(return_lookback)
        cov_list.append(covs)

    df_cov = pd.DataFrame({"date": df.date.unique()[lookback:], "cov_list": cov_list, "return_list": return_list})
    # df["covariance_matrix"] = df_cov
    # df = df.sort_values(["date", "tic"]).reset_index(drop=True)
    return df_cov


# TODO: fixme add covariance as states
df_cov = add_covariance_as_nested_states(data_features.copy())
# TODO: merge to dataset

  0%|          | 0/2768 [08:27<?, ?it/s]


KeyboardInterrupt: 

In [66]:
# print(df_cov.shape)
# print(df_cov['cov_list'].shape)
# print(df_cov['return_list'].shape)

# df_cov['cov_list'] = df_cov['cov_list'].apply(lambda x: pd.DataFrame(x))
# df_cov['cov_list']
# _d3 = pd.DataFrame(df_cov['cov_list'][0])
# _d4 = df_cov['cov_list']
# _d3 = pd.DataFrame(df_cov['return_list'][0])
# _d4 = df_cov['return_list'][0]
# df_cov['return_list']
# _d1 = df_cov['cov_list']
# _d2 = df_cov['return_list']

(2768, 3)
(2768,)
(2768,)


In [40]:
df_cov.to_json("foo.json")

In [26]:

df_cov1 = pd.DataFrame({"cov_list":})


0       [[0.0002849618759954093, 9.196035527740299e-05...
1       [[0.0002850066689925423, 9.224443498717904e-05...
2       [[0.0002838971487853813, 9.16941940901012e-05,...
3       [[0.0002838698218188037, 9.158961530444638e-05...
4       [[0.00028388999474777423, 9.158103382271378e-0...
                              ...                        
2763    [[0.00025416445352502334, 4.899979240800397e-0...
2764    [[0.00025586787418637315, 4.9525971145642e-05,...
2765    [[0.0002513291755320819, 4.902713173926194e-05...
2766    [[0.00025048848339192836, 4.9832109315140924e-...
2767    [[0.0002503516522709732, 5.0038111289378386e-0...
Name: cov_list, Length: 2768, dtype: object

In [30]:
# Create a dataframe with some data
data = pd.DataFrame([1, 2, 3, 4, 5])
print(data)

# Create an empty dataframe
df = pd.DataFrame({"a": [5, 5, 5, 5, 5]})

# Put the dataframe into a cell in the empty dataframe
df.at[0] = data

# Print the resulting dataframe
print(df)

   0
0  1
1  2
2  3
3  4
4  5


InvalidIndexError: You can only assign a scalar value not a <class 'pandas.core.frame.DataFrame'>

In [17]:
# _df = data_features.copy()

# rolling_window =
# cov_matrix = _df.rolling(window=252).cov()

# print(rolling_window)
# print(_df.shape)

# data_features_cov2 = cov2(data_features.copy())

In [18]:
# print(data_features_cov2.shape)

In [37]:
print_info(data_processor.dataframe)

df.shape=(0, 0)


KeyError: 'date'

In [29]:
data_features_cov1.head()

,date,open,high,low,close,adjusted_close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2011-01-03,11.630000,11.795000,11.601429,11.770357,10.033262,445138400,AAPL,0,0.118248,11.709899,11.326780,62.862180,144.286559,25.488753,11.415357,11.212333,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
1,2011-01-03,55.200001,56.279999,55.180000,55.549999,41.640541,5453300,AMGN,0,0.261720,58.568809,52.495191,50.863117,34.401812,4.927279,54.949000,55.565166,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
2,2011-01-03,43.299999,43.619999,43.110001,43.400002,36.488426,7633300,AXP,0,-0.178628,47.098241,41.260759,51.443736,-26.423899,13.049417,43.883000,42.473000,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
3,2011-01-03,66.150002,66.680000,66.000000,66.400002,53.034523,8072900,BA,0,-0.081580,66.520526,63.197475,51.659856,149.651676,11.924376,64.853001,66.948501,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
4,2011-01-03,94.379997,94.809998,94.110001,94.150002,68.190575,5231500,CAT,0,2.266966,96.271975,88.886025,68.687627,82.330220,50.929600,90.157333,85.266833,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...


In [15]:
_d = data_features_cov1["cov_list"][0]
_d

NameError: name 'data_features_cov1' is not defined

In [20]:
# check_dir()

save_dataset(dataframe=data_features_cov1, type=FileType.JSON, filename)

# df_json = pd.read_json(filename, orient="split")
# df_json.head()

Saving data into: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-14T20-13-55.json


KeyboardInterrupt: 